In [ ]:
#-----------------------------------------------------------
#CALCULANDO EN NUMERO LOS DIAS
#-----------------------------------------------------------
import pandas as pd

# Fecha inicial del dataset
fecha_inicio = pd.Timestamp("2026-01-01 00:00")

# Pedir fecha al usuario
fecha_input = input("Ingresa la fecha (YYYY-MM-DD HH:MM): ")

# Convertir a Timestamp
fecha_objetivo = pd.Timestamp(fecha_input)

# Calcular diferencia en horas
horas = (fecha_objetivo - fecha_inicio).total_seconds() / 3600

# Convertir a índice (cada 6 horas)
indice = int(horas / 6)

print(f"La fecha {fecha_objetivo} corresponde al time = {indice}")



In [ ]:
#-----------------------------------------------------------
##-----------------------------------------------------------
#CODIGO PARA CALCULAR MAPAS A DIFERENTES ALTURAS GEOPOTENCIALES
#-----------------------------------------------------------

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


ds = xr.open_dataset(r"C:\Users\User\Desktop\hgt\hgt.nc")
print(ds)

print(ds.time.values[:5])

niveles = [850, 500, 200]

for nivel in niveles:
    gph = (
        ds["hgt"]
        .sel(level=nivel)
        .isel(time=slice(296, 356))
        .mean(dim="time")
        / 9.80665
    )

    if gph.lon.max() > 180:
        gph = gph.assign_coords(lon=(((gph.lon + 180) % 360) - 180))
        gph = gph.sortby("lon")

    gph = gph.sel(
        lat=slice(15, -60),
        lon=slice(-90, -30)
    )

    lons = gph["lon"].values
    lats = gph["lat"].values
    data = gph.values
    LON, LAT = np.meshgrid(lons, lats)

    plt.figure(figsize=(9,5))
    ax = plt.axes(projection=ccrs.PlateCarree())

    pcm = ax.pcolormesh(
        LON, LAT, data,
        transform=ccrs.PlateCarree(),
        shading="nearest",
        cmap="Spectral_r"

    )

    plt.colorbar(pcm, ax=ax, label="Altura geopotencial (m)")
    ax.contour(LON, LAT, data, levels=10, colors="black")
    ax.coastlines()
    ax.gridlines(draw_labels=True,
    linewidth=1,
    color="white",
    alpha=0.4,
    linestyle="--"
    )

    ax.set_title(f"Altura geopotencial media en {nivel} hPa | 16–30 marzo 2017",
    loc="center",
    fontsize=12
    )

    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.STATES, linewidth=0.3)
    ax.set_extent([-90, -30, -60, 15], crs=ccrs.PlateCarree())
    
    # Definir intervalos por nivel
    intervalos = {850:20, 500:30, 200:20}

    # Contornos
    min_val = np.floor(data.min()/intervalos[nivel])*intervalos[nivel]
    max_val = np.ceil(data.max()/intervalos[nivel])*intervalos[nivel]
    levels = np.arange(min_val, max_val+intervalos[nivel], intervalos[nivel])

    cs = ax.contour(
        LON, LAT, data,
        levels=levels,
        colors="black",
        linewidths=1.2,
        transform=ccrs.PlateCarree()
    )

    # Etiquetar isoípsas
    ax.clabel(cs, fmt="%d", fontsize=9)

    plt.show()


In [ ]:
# ==========================================
# MAPA DE PRESIÓN A NIVEL DEL MAR (MSLP) - PROMEDIO
# ==========================================

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# ------------------------------------------
# 1. ABRIR ARCHIVO
# ------------------------------------------
ds = xr.open_dataset(r"C:\Users\User\Desktop\hgt\slp.nc")
print(ds)

# ------------------------------------------
# 2. SELECCIONAR VARIABLE
# ------------------------------------------
for v in ["slp", "msl", "prmsl"]:
    if v in ds.variables:
        mslp = ds[v]
        print(f"Usando variable: {v}")
        break
else:
    raise ValueError("No se encontró presión a nivel del mar")

# ------------------------------------------
# 3. PROMEDIO SOBRE TIEMPO
# ------------------------------------------
# Ajusta el slice según los timesteps que quieras promediar
mslp_mean = mslp.isel(time=slice(296, 356)).mean(dim="time", skipna=True)

# Convertir a numpy 2D limpio para evitar problemas con coords de xarray
mslp_plot = mslp_mean.values

# ------------------------------------------
# 4. DETECTAR UNIDADES AUTOMÁTICAMENTE
# ------------------------------------------
units = mslp.attrs.get("units", "").lower()

if "pa" in units:
    mslp_plot = mslp_plot / 100.0
    print("Unidades detectadas: Pa → convertidas a hPa")
else:
    print("Unidades detectadas: hPa (sin conversión)")

# ------------------------------------------
# 5. DETECTAR LAT / LON
# ------------------------------------------
lat_name = "lat" if "lat" in mslp.coords else "latitude"
lon_name = "lon" if "lon" in mslp.coords else "longitude"

lat = mslp[lat_name].values
lon = mslp[lon_name].values

# ------------------------------------------
# 6. DEFINIR NIVELES DINÁMICOS
# ------------------------------------------
vmin = float(np.nanmin(mslp_plot))
vmax = float(np.nanmax(mslp_plot))

print(f"Rango MSLP: {vmin:.1f} – {vmax:.1f} hPa")

levels = np.arange(
    np.floor(vmin/4)*4,
    np.ceil(vmax/4)*4 + 4,
    4
)

# ------------------------------------------
# 7. MAPA
# ------------------------------------------
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent([-90, -30, -60, 15], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.COASTLINE, linewidth=1)
ax.add_feature(cfeature.BORDERS, linewidth=0.6)
ax.add_feature(cfeature.LAND, facecolor="lightgray")
ax.add_feature(cfeature.OCEAN, facecolor="white")

# ------------------------------------------
# 8. ISOBARAS
# ------------------------------------------
cs = ax.contour(
    lon,
    lat,
    mslp_plot,          # numpy 2D
    levels=levels,
    colors="blue",
    linewidths=1,
    transform=ccrs.PlateCarree()
)

ax.clabel(cs, fmt="%d", fontsize=9)

cfill = ax.contourf(
    lon,
    lat,
    mslp_plot,
    levels=levels,
    cmap="coolwarm",  # o "viridis", "RdBu_r"
    extend="both",
    transform=ccrs.PlateCarree()
)
plt.colorbar(cfill, ax=ax, orientation="vertical", label="hPa")

ax.gridlines(draw_labels=True,
linewidth=1,
color="white",
alpha=0.4,
linestyle="--"
)

plt.title("Presión a Nivel del Mar media | 16–30 marzo 2017", fontsize=14)
loc="center",
fontsize=12
    
plt.show()